[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/ai-rag-lab/)


# Step 1: Setup prerequisites

In [88]:
import os
from pymongo import MongoClient

In [89]:
# If you are using your own MongoDB Atlas cluster, use the connection string for your cluster here
MONGODB_URI = os.environ.get("MONGODB_URI")
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI, appname="devrel.workshop.rag")
# Check the connection to the server
mongodb_client.admin.command("ping")

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1747247951, 1),
  'signature': {'hash': b'\xdfC\xa1\xc2\x9f>B%Pu)Q\xa6x\xe4\xd7y\x11E5',
   'keyId': 7504308584902033414}},
 'operationTime': Timestamp(1747247951, 1)}

In [90]:
SERVERLESS_URL = os.environ.get("SERVERLESS_URL")

# Step 2: Download the dataset

In [91]:
# You may see a warning upon running this cell. You can ignore it.
import pandas as pd
from datasets import load_dataset

In [92]:
# Download the `mongodb-docs` dataset from Hugging Face
data = load_dataset("mongodb/mongodb-docs", split="train")
# Convert the dataset into a dataframe first, then into a list of Python objects/dictionaries
docs = pd.DataFrame(data).to_dict("records")

In [93]:
# Note the number of documents in the dataset
len(docs)

20

In [94]:
# Preview a document to understand its structure
docs[0]

{'updated': '2024-05-20T17:30:49.148Z',
 'metadata': {'contentType': None,
  'productName': 'MongoDB Atlas',
  'tags': ['atlas', 'docs'],
  'version': None},
 'action': 'created',
 'sourceName': 'snooty-cloud-docs',
 'body': '# View Database Access History\n\n- This feature is not available for `M0` free clusters, `M2`, and `M5` clusters. To learn more, see Atlas M0 (Free Cluster), M2, and M5 Limits.\n\n- This feature is not supported on Serverless instances at this time. To learn more, see Serverless Instance Limitations.\n\n## Overview\n\nAtlas parses the MongoDB database logs to collect a list of authentication requests made against your clusters through the following methods:\n\n- `mongosh`\n\n- Compass\n\n- Drivers\n\nAuthentication requests made with API Keys through the Atlas Administration API are not logged.\n\nAtlas logs the following information for each authentication request within the last 7 days:\n\n<table>\n<tr>\n<th id="Field">\nField\n\n</th>\n<th id="Description">\nD

# Step 3: Chunk up the data


In [95]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Dict, List

In [96]:
# Common list of separators for text data
separators = ["\n\n", "\n", " ", "", "#", "##", "###"]

In [97]:
# Use the `RecursiveCharacterTextSplitter` from LangChain to first split a piece of text on the list of `separators` above.
# Then recursively merge them into tokens until the specified chunk size is reached.
# For text data, you typically want to keep 1-2 paragraphs (~200 tokens) in a single chunk.
# Chunk overlap of 15-20% of the chunk size is recommended to maintain context between chunks.
# The `model_name` parameter indicates which encoder to use for tokenization, in this case GPT-4's encoder.
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4", separators=separators, chunk_size=200, chunk_overlap=30
)

📚 https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In [98]:
def get_chunks(doc: Dict, text_field: str) -> List[Dict]:
    """
    Chunk up a document.

    Args:
        doc (Dict): Parent document to generate chunks from.
        text_field (str): Text field to chunk.

    Returns:
        List[Dict]: List of chunked documents.
    """
    # Extract the field to chunk from `doc`
    text = doc[text_field]
    # Split `text` using the `split_text` method of the `text_splitter` object
    # NOTE: `text` is a string
    chunks = text_splitter.split_text(text)

    # Iterate through `chunks` and for each chunk:
    # 1. Create a shallow copy of `doc`, call it `temp`
    # 2. Set the `text_field` field in `temp` to the content of the chunk
    # 3. Append `temp` to `chunked_data`
    chunked_data = []
    for chunk in chunks:
        temp = doc.copy()
        temp[text_field]=chunk
        chunked_data.append(temp)

    return chunked_data

In [99]:
split_docs = []
# Iterate through `docs`, use the `get_chunks` function to chunk up the "body" field in the documents, and add the list of chunked documents to `split_docs` initialized above.
for doc in docs:
    chunks =get_chunks(doc, "body")
    split_docs.extend(chunks)

In [100]:
# Notice that the length of `split_docs` is greater than the length of `docs` from Step 2 above
# This is because each document in `docs` has been split into multiple chunks
len(split_docs)

107

In [101]:
# Preview a chunked document to understand its structure
# Note that the structure looks similar to the original docs, except the `body` field now contains smaller chunks of text
split_docs[0]

{'updated': '2024-05-20T17:30:49.148Z',
 'metadata': {'contentType': None,
  'productName': 'MongoDB Atlas',
  'tags': ['atlas', 'docs'],
  'version': None},
 'action': 'created',
 'sourceName': 'snooty-cloud-docs',
 'body': '# View Database Access History\n\n- This feature is not available for `M0` free clusters, `M2`, and `M5` clusters. To learn more, see Atlas M0 (Free Cluster), M2, and M5 Limits.\n\n- This feature is not supported on Serverless instances at this time. To learn more, see Serverless Instance Limitations.\n\n## Overview\n\nAtlas parses the MongoDB database logs to collect a list of authentication requests made against your clusters through the following methods:\n\n- `mongosh`\n\n- Compass\n\n- Drivers\n\nAuthentication requests made with API Keys through the Atlas Administration API are not logged.\n\nAtlas logs the following information for each authentication request within the last 7 days:\n\n<table>\n<tr>\n<th id="Field">\nField\n\n</th>\n<th id="Description">\nD

# Step 4: Generate embeddings


In [102]:
# You may see a warning upon running this cell. You can ignore it.
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [103]:
# Load the `gte-small` model using the Sentence Transformers library
embedding_model = SentenceTransformer("thenlper/gte-small")

📚 https://huggingface.co/thenlper/gte-small#usage (See "Use with sentence-transformers" under Usage)

In [104]:
# Define a function that takes a piece of text (`text`) as input, embeds it using the `embedding_model` instantiated above and returns the embedding as a list
# An array can be converted to a list using the `tolist()` method
def get_embedding(text: str) -> List[float]:
    """
    Generate the embedding for a piece of text.

    Args:
        text (str): Text to embed.

    Returns:
        List[float]: Embedding of the text as a list.
    """
    embedding = embedding_model.encode(text)
    return embedding.tolist()

In [105]:
embedded_docs = []
# Add an `embedding` field to each dictionary in `split_docs`
# The `embedding` field should correspond to the embedding of the value of the `body` field
# Use the `get_embedding` function defined above to generate the embedding
# Append the updated dictionaries to `embedded_docs` initialized above.
for doc in tqdm(split_docs):
    doc["embedding"] = get_embedding(doc["body"])
    embedded_docs.append(doc)

100%|██████████| 107/107 [00:07<00:00, 13.54it/s]


In [106]:
# Check that the length of `embedded_docs` is the same as that of `split_docs`
len(embedded_docs)

107

# Step 5: Ingest data into MongoDB


### **Do not change the values assigned to the variables below**

In [107]:
# Name of the database -- Change if needed or leave as is
DB_NAME = "mongodb_genai_devday_rag"
# Name of the collection -- Change if needed or leave as is
COLLECTION_NAME = "knowledge_base"
# Name of the vector search index -- Change if needed or leave as is
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

In [108]:
# Connect to the `COLLECTION_NAME` collection.
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [109]:
# Bulk delete all existing records from the collection defined above
collection.delete_many({})

DeleteResult({'n': 107, 'electionId': ObjectId('7fffffff0000000000000005'), 'opTime': {'ts': Timestamp(1747247970, 11), 't': 5}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747247970, 11), 'signature': {'hash': b'V\x81\x19\xc6\xe4\x96\x1bs\xe5j\x0c"\xd7\x04\x0b\xeb\xa2\xd3b\xfa', 'keyId': 7504308584902033414}}, 'operationTime': Timestamp(1747247970, 11)}, acknowledged=True)

📚 https://pymongo.readthedocs.io/en/stable/examples/bulk.html#bulk-insert


In [110]:
# Bulk insert `embedded_docs` into the `collection` defined above -- should be a one-liner
collection.insert_many(embedded_docs)

print(f"Ingested {collection.count_documents({})} documents into the {COLLECTION_NAME} collection.")

Ingested 107 documents into the knowledge_base collection.


# Step 6: Create a vector search index

In [111]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 384,
                "similarity": "cosine",
            }
        ]
    },
}

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_search_index

In [114]:
# Create a vector search index with the above definition for the `collection` collection
collection.create_search_index(model=model)

OperationFailure: Index vector_index already exists with a different definition. Drop it first if needed., full error: {'ok': 0.0, 'errmsg': 'Index vector_index already exists with a different definition. Drop it first if needed.', 'code': 68, 'codeName': 'IndexAlreadyExists', '$clusterTime': {'clusterTime': Timestamp(1747247970, 14), 'signature': {'hash': b'V\x81\x19\xc6\xe4\x96\x1bs\xe5j\x0c"\xd7\x04\x0b\xeb\xa2\xd3b\xfa', 'keyId': 7504308584902033414}}, 'operationTime': Timestamp(1747247970, 14)}

In [ ]:
# Verify the index was created and is in READY status before proceeding
list(collection.list_search_indexes())

[{'id': '6824ccf320f21b6923dcc63f',
  'name': 'vector_index',
  'type': 'vectorSearch',
  'status': 'READY',
  'queryable': True,
  'latestVersion': 0,
  'latestDefinition': {'fields': [{'type': 'vector',
     'path': 'embedding',
     'numDimensions': 384,
     'similarity': 'cosine'}]}}]

# Step 7: Perform vector search on your data


### Define a vector search function

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Basic Example")


In [115]:
# Define a function to retrieve relevant documents for a user query using vector search
def vector_search(user_query: str) -> List[Dict]:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query string.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the `user_query` using the `get_embedding` function defined in Step 4
    query_embedding = get_embedding(user_query)

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 150 and only return the top 5 documents from the vector search
    # In the $project stage, exclude the `_id` field and include only the `body` field and `vectorSearchScore`
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = [
    {
        "$vectorSearch": {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,
            "limit": 5
        }
    },
    {
        "$project": {
            "_id": 0,
            "body": 1,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

    # Execute the aggregation `pipeline` and store the results in `results`
    results = collection.aggregate(pipeline)
    return list(results)

### Run vector search queries


In [116]:
vector_search("What are some best practices for data backups in MongoDB?")

[{'body': '# Backup and Restore Sharded Clusters\n\nThe following tutorials describe backup and restoration for sharded clusters:\n\nTo use `mongodump` and `mongorestore` as a backup strategy for sharded clusters, you must stop the sharded cluster balancer and use the `fsync` command or the `db.fsyncLock()` method on `mongos` to block writes on the cluster during backups.\n\nSharded clusters can also use one of the following coordinated backup and restore processes, which maintain the atomicity guarantees of transactions across shards:\n\n- MongoDB Atlas\n\n- MongoDB Cloud Manager\n\n- MongoDB Ops Manager\n\nUse file system snapshots back up each component in the sharded cluster individually. The procedure involves stopping the cluster balancer. If your system configuration allows file system backups, this might be more efficient than using MongoDB tools.\n\nCreate backups using `mongodump` to back up each component in the cluster individually.',
  'score': 0.9431126117706299},
 {'body

In [117]:
vector_search("How to resolve alerts in MongoDB?")

[{'body': '### Query Targeting\n\nQuery Targeting alerts often indicate inefficient queries.\n\nTo learn more, see Fix Query Issues.\n\n### Connection Limits\n\nConnection alerts typically occur when the maximum number of allowable connections to a MongoDB process has been exceeded. Once the limit is exceeded, no new connections can be opened until the number of open connections drops down below the limit.\n\nTo learn more, see Fix Connection Issues.\n\n## Alerts Workflow\n\nWhen an alert condition is met, the alert lifecycle begins.\n\nTo learn more, see the Alerts Workflow.',
  'score': 0.9637914896011353},
 {'body': '# Resolve Alerts\n\nAtlas issues alerts for the database and server conditions configured in your alert settings. When a condition triggers an alert, Atlas displays a warning symbol on the cluster and sends alert notifications. Your alert settings determine the notification methods. Atlas continues sending notifications at regular intervals until the condition resolves 

# 🦹‍♀️ Combine pre-filtering with vector search

### Filter for documents where the product name is `MongoDB Atlas`

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [118]:
# Modify the vector search index `model` from Step 6 to include the `metadata.productName` field as a `filter` field
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 384,
                "similarity": "cosine"
            },
            {"type": "filter", "path": "metadata.productName"}
        ]
    }
}

In [119]:
# Re-create the vector search index with the modified model
collection.drop_search_index(name=ATLAS_VECTOR_SEARCH_INDEX_NAME)
collection.create_search_index(model=model)

'vector_index'

In [120]:
# Verify that the index has the right filter fields and is in READY status before proceeding
list(collection.list_search_indexes())

[{'id': '6824e38719cc1c4cc59d95f9',
  'name': 'vector_index',
  'type': 'vectorSearch',
  'status': 'READY',
  'queryable': True,
  'latestVersion': 0,
  'latestDefinition': {'fields': [{'type': 'vector',
     'path': 'embedding',
     'numDimensions': 384,
     'similarity': 'cosine'},
    {'type': 'filter', 'path': 'metadata.productName'}]}}]

In [121]:
# Embed the user query
query_embedding = get_embedding(
    "What are some best practices for data backups in MongoDB?"
)

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Filter Example")

In [122]:
# Modify the aggregation pipeline defined in Step 7 to:
# Include a filter in the $vectorSearch stage for documents where the `metadata.productName` field has the value "MongoDB Atlas".
# Include the `metadata.productName` in the $project stage of the pipeline.
pipeline = [
    {
        "$vectorSearch": {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "path": "embedding",
            "queryVector": query_embedding,
            "numCandidates": 150,
            "limit": 5,
            "filter": {"metadata.productName": "MongoDB Atlas"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "body": 1,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

In [123]:
# Execute the aggregation pipeline and view the results
results = collection.aggregate(pipeline)
list(results)

[{'body': '# View Database Access History\n\n- This feature is not available for `M0` free clusters, `M2`, and `M5` clusters. To learn more, see Atlas M0 (Free Cluster), M2, and M5 Limits.\n\n- This feature is not supported on Serverless instances at this time. To learn more, see Serverless Instance Limitations.\n\n## Overview\n\nAtlas parses the MongoDB database logs to collect a list of authentication requests made against your clusters through the following methods:\n\n- `mongosh`\n\n- Compass\n\n- Drivers\n\nAuthentication requests made with API Keys through the Atlas Administration API are not logged.\n\nAtlas logs the following information for each authentication request within the last 7 days:\n\n<table>\n<tr>\n<th id="Field">\nField\n\n</th>\n<th id="Description">\nDescription\n\n</th>\n</tr>\n<tr>\n<td headers="Field">\nTimestamp',
  'score': 0.9098457098007202},
 {'body': '### Query Targeting\n\nQuery Targeting alerts often indicate inefficient queries.\n\nTo learn more, see 

### Filter on documents which have been updated on or after `2024-05-19` and where the content type is `Tutorial`

In [124]:
# Modify the vector search index `model` from Step 6 to include `metadata.contentType` and `updated` as `filter` fields
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 384,
                "similarity": "cosine"
            },
            {"type": "filter", "path": "metadata.contentType"},
            {"type": "filter", "path": "updated"}
        ]
    }
}

In [125]:
# Re-create the vector search index with the modified model
collection.drop_search_index(name=ATLAS_VECTOR_SEARCH_INDEX_NAME)
collection.create_search_index(model=model)

'vector_index'

In [126]:
# Verify that the index has the right filter fields and is in READY status before proceeding
list(collection.list_search_indexes())

[{'id': '6824e39919cc1c4cc59d95fa',
  'name': 'vector_index',
  'type': 'vectorSearch',
  'status': 'READY',
  'queryable': True,
  'latestVersion': 0,
  'latestDefinition': {'fields': [{'type': 'vector',
     'path': 'embedding',
     'numDimensions': 384,
     'similarity': 'cosine'},
    {'type': 'filter', 'path': 'metadata.contentType'},
    {'type': 'filter', 'path': 'updated'}]}}]

In [ ]:
# Embed the user query
query_embedding = get_embedding(
    "What are some best practices for data backups in MongoDB?"
)

In [ ]:
# Modify the aggregation pipeline defined in Step 7 to:
# Include a filter in the $vectorSearch stage for documents where the `metadata.contentType` field is "Tutorial" AND the `updated` field is greater than or equal to "2024-05-19".
# Include the `metadata.contentType` and `updated` fields in the $project stage of the pipeline.
pipeline = [
    {
        "$vectorSearch": {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "path": "embedding",
            "queryVector": query_embedding,
            "numCandidates": 150,
            "limit": 5,
            "filter": {
                "$and": [
                    {"metadata.contentType": "Tutorial"},
                    {"updated": {"$gte": "2024-05-19"}}
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "body": 1,
            "updated": 1,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

In [ ]:
# Execute the aggregation pipeline and view the results
results = collection.aggregate(pipeline)
list(results)

[{'updated': '2024-05-20T17:32:23.500Z',
  'body': '**Getting the best recommendations**\n\nFor best results, follow these practices.\n\n - Give CodeWhisperer something to work with. The more code your file contains, the more context CodeWhisperer has for generating recommendations.\n - Write descriptive comments in natural language — for example\n```\n// Take a JSON document as a String and store it in MongoDB returning the _id\n```\nOr\n```\n//Insert a document in a collection with a given _id and a discountLevel\n```\n - Specify the libraries you prefer at the start of your file by using import statements.\n```\n// This Java class works with MongoDB sync driver.\n// This class implements Connection to MongoDB and CRUD methods.\n```\n - Use descriptive names for variables and functions\n - Break down complex tasks into simpler tasks\n\n**Provide feedback**\n----------------',
  'score': 0.9205133318901062},
 {'updated': '2024-05-20T17:32:23.500Z',
  'body': "# Getting Started with Mo

# Step 8: Build the RAG application


In [ ]:
import requests

### Define a function to create the chat prompt

In [ ]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = vector_search(user_query)
    # Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
    context = "\n\n".join([doc.get('body') for doc in context])
    # Prompt consisting of the question and relevant context to answer it
    prompt = f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}\n\nQuestion:{user_query}"
    return prompt

### Define a function to answer user queries

In [ ]:
# Define a function to answer user queries
def generate_answer(user_query: str) -> None:
    """
    Generate an answer to the user query.

    Args:
        user_query (str): The user's query string.
    """
    # Use the `create_prompt` function above to create a chat prompt
    prompt = create_prompt(user_query)
    # Format the message to the LLM in the format [{"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # Use the `prompt` created above to populate the `content` field in the chat message
    messages = [{"role": "user", "content": prompt}]
    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})
    # Print the final answer
    print(response.json()["text"])

### Query the RAG application


In [ ]:
generate_answer("What are some best practices for data backups in MongoDB?")

Based on the provided context, some best practices for data backups in MongoDB sharded clusters include:

1. Stopping the sharded cluster balancer before performing backups.

2. Using the `fsync` command or `db.fsyncLock()` method on `mongos` to block writes on the cluster during backups when using `mongodump` and `mongorestore`.

3. Using coordinated backup and restore processes that maintain atomicity guarantees across shards, such as MongoDB Atlas, MongoDB Cloud Manager, or MongoDB Ops Manager.

4. Using file system snapshots to back up each component in the sharded cluster individually, if the system configuration allows it.

5. Using `mongodump` to back up each component in the cluster individually.

6. Limiting the operation of the cluster balancer to provide a window for regular backup operations.

7. For full sharded cluster backups, following a specific procedure and considerations for restoring the entire cluster from backup.

These practices help ensure consistent and reliab

In [ ]:
# Notice that the LLM does not remember the conversation history at this stage
generate_answer("What did I just ask you?")

I DON'T KNOW. The context provided does not contain any question that you asked me. The context appears to be information about using CodeWhisperer and some JSON data, but there is no previous question mentioned.


# 🦹‍♀️ Re-rank retrieved results


In [ ]:
from sentence_transformers import CrossEncoder

In [ ]:
rerank_model = CrossEncoder("mixedbread-ai/mxbai-rerank-xsmall-v1")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


📚 https://huggingface.co/mixedbread-ai/mxbai-rerank-xsmall-v1#quickstart

In [ ]:
# Add a re-ranking step to the following function
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = vector_search(user_query)
    # Extract the "body" field from each document in `context`
    documents = [d.get("body") for d in context]
    # Use the `rerank_model` instantiated above to re-rank `documents`
    # Set the `top_k` argument to 5 
    reranked_documents = rerank_model.rank(
    user_query, documents, return_documents=True, top_k=5
)
    # Join the re-ranked documents into a single string, where each document is separated by two new lines ("\n\n")
    context = "\n\n".join([d.get("text", "") for d in reranked_documents])
    # Prompt consisting of the question and relevant context to answer it
    prompt = f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}\n\nQuestion:{user_query}"
    return prompt

In [ ]:
# Note the impact of re-ranking on the generated answer
# You might not see a difference in this example since we are only re-ranking 5 documents
# In practice, you would send a larger number of documents to the re-ranker, and get the top few AFTER reranking
generate_answer("What are some best practices for data backups in MongoDB?")

Based on the provided context, some best practices for data backups in MongoDB sharded clusters include:

1. Stopping the sharded cluster balancer before performing backups.

2. Using the `fsync` command or `db.fsyncLock()` method on `mongos` to block writes during backups when using `mongodump` and `mongorestore`.

3. Using coordinated backup and restore processes that maintain atomicity guarantees across shards, such as MongoDB Atlas, MongoDB Cloud Manager, or MongoDB Ops Manager.

4. Using file system snapshots to back up each component in the sharded cluster individually, if the system configuration allows.

5. Using `mongodump` to back up each component in the cluster individually.

6. Limiting the operation of the cluster balancer to provide a window for regular backup operations.

7. For full sharded cluster backups, following a specific procedure and considerations for restoring the entire cluster.

The context also mentions that it's important to consider the Full Time Diagnos

# Step 9: Add memory to the RAG application


In [ ]:
from datetime import datetime

In [ ]:
history_collection = mongodb_client[DB_NAME]["chat_history"]

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_index


In [ ]:
# Create an index on the key `session_id` for the `history_collection` collection
history_collection.create_index("session_id")

'session_id_1'

### Define a function to store chat messages in MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.insert_one

In [127]:
def store_chat_message(session_id: str, role: str, content: str) -> None:
    """
    Store a chat message in a MongoDB collection.

    Args:
        session_id (str): Session ID of the message.
        role (str): Role for the message. One of `system`, `user` or `assistant`.
        content (str): Content of the message.
    """
    # Create a message object with `session_id`, `role`, `content` and `timestamp` fields
    # `timestamp` should be set the current timestamp
    message = {
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.now(),
    }
    # Insert the `message` into the `history_collection` collection
    history_collection.insert_one(message)

### Define a function to retrieve chat history from MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort

In [ ]:
def retrieve_session_history(session_id: str) -> List:
    """
    Retrieve chat message history for a particular session.

    Args:
        session_id (str): Session ID to retrieve chat message history for.

    Returns:
        List: List of chat messages.
    """
    # Query the `history_collection` collection for documents where the "session_id" field has the value of the input `session_id`
    # Sort the results in increasing order of the values in `timestamp` field
    cursor =  history_collection.find({"session_id": session_id}).sort("timestamp", 1)

    if cursor:
        # Iterate through the cursor and extract the `role` and `content` field from each entry
        # Then format each entry as: {"role": <role_value>, "content": <content_value>}
        messages = [{"role": msg["role"], "content": msg["content"]} for msg in cursor]
    else:
        # If cursor is empty, return an empty list
        messages = []

    return messages

### Handle chat history in the `generate_answer` function

In [ ]:
def generate_answer(session_id: str, user_query: str) -> None:
    """
    Generate an answer to the user's query taking chat history into account.

    Args:
        session_id (str): Session ID to retrieve chat history for.
        user_query (str): The user's query string.
    """
    # Initialize list of messages to pass to the chat completion model
    messages = []

    # Retrieve documents relevant to the user query and convert them to a single string
    context = vector_search(user_query)
    context = "\n\n".join([d.get("body", "") for d in context])
    # Create a system prompt containing the retrieved context
    system_message = {
        "role": "user",
        "content": f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}",
    }
    # Append the system prompt to the `messages` list
    messages.append(system_message)

    # Use the `retrieve_session_history` function to retrieve message history from MongoDB for the session ID `session_id` 
    # And add all messages in the message history to the `messages` list 
    message_history = retrieve_session_history(session_id)
    messages.extend(message_history)

    # Format the user query in the format {"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # And append the user message to the `messages` list
    user_message = {"role": "user", "content": user_query}
    messages.append(user_message)

    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})

    # Extract the answer from the response
    answer = response.json()["text"]

    # Use the `store_chat_message` function to store the user message and also the generated answer in the message history collection
    # The role value for user messages is "user", and "assistant" for the generated answer
    store_chat_message(session_id, "user", user_query)
    store_chat_message(session_id, "assistant", answer)
    print(answer)

In [ ]:
generate_answer(
    session_id="1",
    user_query="What are some best practices for data backups in MongoDB?",
)

Based on the provided context, some best practices for data backups in MongoDB sharded clusters include:

1. Stopping the sharded cluster balancer before performing backups.

2. Using the fsync command or db.fsyncLock() method on mongos to block writes during backups when using mongodump and mongorestore.

3. Using coordinated backup and restore processes that maintain transaction atomicity across shards, such as MongoDB Atlas, MongoDB Cloud Manager, or MongoDB Ops Manager.

4. Using file system snapshots to back up each component of the sharded cluster individually, if the system configuration allows it.

5. Using mongodump to back up each component of the cluster individually.

6. Limiting the operation of the cluster balancer to provide a window for regular backup operations.

7. For full cluster backups, following a specific procedure to restore the entire sharded cluster from backup.

The context also mentions that it's important to consider the atomicity guarantees of transaction

In [128]:
generate_answer(
    session_id="1",
    user_query="What did I just ask you?",
)

I apologize for the confusion in my previous response. You actually asked:

"What did I just ask you?"

I made a mistake by answering a different question that wasn't actually asked. Thank you for bringing this to my attention, and I'll strive to be more accurate in the future.
